In [1]:
import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd
import datetime
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import urllib.request

In [11]:
today = datetime.datetime.now()

My_API_Key = unquote('dji8ijRhG7jPHWzskJYswLPTxNia1VySljS9bAguvLozhy5b9CRCalaZ%2BJIU647gp%2BzEyVF%2Bbv%2FGQukkiBk6qQ%3D%3D')  

params =    {
    quote_plus('ServiceKey') : My_API_Key,
       # quote_plus('pageNo') : '1',
       # quote_plus('numOfRows') : '10',
        quote_plus('dataType') : 'XML',
        quote_plus('base_date') : today.strftime('%Y%m%d'),
        quote_plus('base_time') : '0500',
        quote_plus('nx') : '59',
        quote_plus('ny') : '74'}


url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst'
res = requests.get(url, params=params)
soup = bs(res.text, 'lxml')

items = soup.find_all('item')


# 응답 메시지 

#XML을 딕셔너리로 변환 
response={
          'category':'카테고리',
          
          'nx':'예보지점 x ',
          'ny':'예보지점 y ',
          'fcstvalue':'예보 값',
          'basetime':'발표시각',
          'fcstdate':'예보일자',
          'basedate':'발표일자',
          'fcsttime':'예보시각',
          '':'',
          '':'',
    
          
          
          
          }


# xml을 dictionary로 바꿔 리스트 원소로 추가
lst = []
for y in items:
    l = {}
    for x in y:
        l[x.name] = x.text
    lst.append(l)

df = pd.DataFrame(lst)
df.columns = df.columns.map(response)
df







발표일자  발표시각 카테고리      예보일자  예보시각  예보 값 예보지점 x  예보지점 y 
0  20210722  0500  POP  20210722  0900     0      59      74
1  20210722  0500  PTY  20210722  0900     0      59      74
2  20210722  0500  REH  20210722  0900    70      59      74
3  20210722  0500  SKY  20210722  0900     1      59      74
4  20210722  0500  T3H  20210722  0900    28      59      74
5  20210722  0500  UUU  20210722  0900  -3.1      59      74
6  20210722  0500  VEC  20210722  0900    95      59      74
7  20210722  0500  VVV  20210722  0900   0.3      59      74
8  20210722  0500  WSD  20210722  0900   3.2      59      74
9  20210722  0500  POP  20210722  1200     0      59      74

In [12]:
#카테고리선정 선정
weatherDf = df.loc[(df['카테고리']=='PTY')]
weather = weatherDf.loc[:,['발표일자','발표시각','카테고리','예보일자','예보시각','예보 값']].dropna()
weather

발표일자  발표시각 카테고리      예보일자  예보시각 예보 값
1  20210722  0500  PTY  20210722  0900    0

In [9]:
'''

# 특정 요소의 코드값 및 범주
- 하늘상태(SKY) 코드 : 맑음(1), 구름많음(3), 흐림(4) 
* 구름조금(2) 삭제 (2019.06.4)
- 강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4), 빗방울(5), 빗방울/눈날림(6), 눈날림(7)
여기서 비/눈은 비와 눈이 섞여 오는 것을 의미 (진눈개비)
* 빗방울(5), 빗방울/눈날림(6), 눈날림(7) 

- 초단기예보, 동네예보 강수량(RN1, R06) 범주 및 표시방법(값)
범주	문자열표시
0.1mm 미만	0mm 또는 없음
0.1mm 이상 1mm 미만	1mm 미만
1 mm 이상 5 mm 미만	1~4mm
5 mm 이상 10 mm 미만	5~9mm
10 mm 이상 20 mm 미만	10~19mm
20 mm 이상 40 mm 미만	20~39mm
40 mm 이상 70 mm 미만	40~69mm
70 mm 이상	70mm 이상
	
'''

'\n\n# 특정 요소의 코드값 및 범주\n- 하늘상태(SKY) 코드 : 맑음(1), 구름많음(3), 흐림(4) \n* 구름조금(2) 삭제 (2019.06.4)\n- 강수형태(PTY) 코드 : 없음(0), 비(1), 비/눈(2), 눈(3), 소나기(4), 빗방울(5), 빗방울/눈날림(6), 눈날림(7)\n여기서 비/눈은 비와 눈이 섞여 오는 것을 의미 (진눈개비)\n* 빗방울(5), 빗방울/눈날림(6), 눈날림(7) \n\n- 초단기예보, 동네예보 강수량(RN1, R06) 범주 및 표시방법(값)\n범주\t문자열표시\n0.1mm 미만\t0mm 또는 없음\n0.1mm 이상 1mm 미만\t1mm 미만\n1 mm 이상 5 mm 미만\t1~4mm\n5 mm 이상 10 mm 미만\t5~9mm\n10 mm 이상 20 mm 미만\t10~19mm\n20 mm 이상 40 mm 미만\t20~39mm\n40 mm 이상 70 mm 미만\t40~69mm\n70 mm 이상\t70mm 이상\n\t\n'

In [10]:
'''
POP	강수확률
PTY	강수형태
R06	6시간 강수량
REH	습도
S06	6시간 신적설
SKY	하늘상태
T3H	3시간 기온
TMN	아침 최저기온
TMX	낮 최고기온
UUU	풍속(동서성분)
VVV	풍속(남북성분)
WAV	파고
VEC	풍향
WSD	풍속
T1H	기온
RN1	1시간 강수량
UUU	동서바람성분
VVV	남북바람성분
REH	습도
PTY	강수형태
VEC	풍향
WSD	풍속
T1H	기온
RN1	1시간 강수량
SKY	하늘상태
UUU	동서바람성분
VVV	남북바람성분
REH	습도
'''

'\nPOP\t강수확률\nPTY\t강수형태\nR06\t6시간 강수량\nREH\t습도\nS06\t6시간 신적설\nSKY\t하늘상태\nT3H\t3시간 기온\nTMN\t아침 최저기온\nTMX\t낮 최고기온\nUUU\t풍속(동서성분)\nVVV\t풍속(남북성분)\nWAV\t파고\nVEC\t풍향\nWSD\t풍속\nT1H\t기온\nRN1\t1시간 강수량\nUUU\t동서바람성분\nVVV\t남북바람성분\nREH\t습도\nPTY\t강수형태\nVEC\t풍향\nWSD\t풍속\nT1H\t기온\nRN1\t1시간 강수량\nSKY\t하늘상태\nUUU\t동서바람성분\nVVV\t남북바람성분\nREH\t습도\n'